In [1]:
import pandas as pd

# first_df = pd.read_excel('RNGF Tag data 11.27.23-12.4.23am.xlsx')
# second_df = pd.read_excel('RNGF Tag data 12.4.23-12.21.23.xlsx')

first_df = pd.read_excel('RNGF Tag data 12.21.23-12.31.23_new.xlsx')
second_df = pd.read_excel('RNGF_Tag_data_Jan_2024.xlsx')
third_df = pd.read_excel('RNGF Tags data 2.1.24-2.7.24.xlsx')
fourth_df = pd.read_excel('RNGF Tag data 11.27.23-12.4.23am.xlsx')
fifth_df = pd.read_excel('RNGF Tag data 12.4.23-12.21.23.xlsx')

In [2]:
combined_df = pd.concat([first_df, second_df, third_df, fourth_df, fifth_df])

In [3]:
combined_df['Start Time'] = pd.to_datetime(combined_df['Start Time'])

### Sort by date


In [4]:
combined_df = combined_df.sort_values(by='Start Time', ascending=True)

### Drop duplicates in the date column

In [5]:
combined_df = combined_df.drop_duplicates(subset='Start Time')

### Reset the master dataframe index

In [6]:
combined_df.reset_index(drop=True, inplace=True)

In [7]:
import sqlite3

connection = sqlite3.connect('combined_df.db')
db = combined_df.to_sql('combined_df', connection, if_exists='replace')



In [8]:
combined_df.columns

Index(['Start Time', 'RNGBLWR-Bstr Blwr 1 Speed FDBK PV- Loop 9145',
       'RNGBLWR-Bstr Blwr 2 Speed FDBK PV- Loop 9149',
       'RNGBLWR-Dig Gas Inlet PRESS PV- Loop 9140',
       'RNGBLWR-Skid Disch FLOW PV- Loop 9159',
       'RNGBLWR-System Gas Disch PRESS PV- Loop 9158',
       'RNGCOMP-Comp Speed PV- Loop 9250',
       'RNGCOMP-Stage 1 Inlet Press PV- Loop 9234',
       'RNGCOMP-Stage 2 Final Press PV- Loop 9241',
       'RNGDGTS-Boigas Stream Flow- Loop 8940',
       'RNGDGTS-Compressor Suction Pressure- Loop 8948',
       'RNGDGTS-Gas flow- Loop 9069',
       'RNGDGTS-Plant compressor Speed- Loop 8964',
       'RNGFLARE-Off-Gas to Flare Flow PV- Loop 9317',
       'RNGFLARE-Reject Gas to Flare Flow PV- Loop 9316',
       'RNGFLARE-VPSA Stg 1 off-gas flare flow- Loop 9315',
       'RNGNWNG-Outlet Pressure- Loop 9341', 'RNGNWNG-RNG Flow- Loop 9327',
       'RNGRTO-Gas Trtmnt Tail Gas Flow PV- Loop 9105',
       'RNGRTO-Inlet PRESSURE PV- Loop 9181',
       'RNGRTO-Iso Valve Clo

### Overview graph data

In [9]:
overview_graph = combined_df[[
    'Start Time', 
    'RNGBLWR-Skid Disch FLOW PV- Loop 9159', # Blower Discharge Flow (scfm) col F
    'RNGBLWR-System Gas Disch PRESS PV- Loop 9158', # Blower Discharge Press (psig) col col G
    'RNGDGTS-Plant compressor Speed- Loop 8964', # Totara Speed (RPM) col CM
    'RNGRTO-Gas Trtmnt Tail Gas Flow PV- Loop 9105.1', # RTO Inlet Flow (scfm) col T
    'RNGRTO-Inlet PRESSURE PV- Loop 9181.1', # RTO Inlet Pressure (in WC) col U
    'RNGFLARE-Reject Gas to Flare Flow PV- Loop 9316', # Reject Gas to Flare (scfm) col P
    'RNGCOMP-Stage 2 Final Press PV- Loop 9241', # ANGI S2 Discharge (psig) col J
    'RNGDGTS-VPSA Stage 1 Running- Loop 9035', # VPSA Status col CW
    ]]





overview_graph_df = pd.DataFrame(overview_graph)
overview_graph_df.columns = [
    'date/time',
    'Blower Discharge Flow (scfm)',
    'Blower Discharge Flow (psig)',
    'Totara Speed (RPM)',
    'RTO Inlet Flow (scfm)',
    'RTO Inlet Pressure (in WC)',
    'Reject Gas to Floar (scfm)',
    'ANGI S2 Discharge (psig)',
    'VPSA Status']
overview_graph_df = overview_graph_df.reset_index(drop=True)
# overview_graph_df.transpose()

overview_graph_df.to_excel('overview_graph_df.xlsx')

### VPSA Overview 

In [ ]:
import numpy as np


vpsa_graph = combined_df[[
    'Start Time',
    'RNGDGTS-VPSA Stage 1 Running- Loop 9035', # VPSA status col CW
    'RNGFLARE-VPSA Stg 1 off-gas flare flow- Loop 9315', # VPSA Stage 1 off-gas flow col Q
    'RNGFLARE-Off-Gas to Flare Flow PV- Loop 9317', # VPSA Stage 2 off-gas flow col O
    'RNGDGTS-VPSA1 Vac Pump P001 Speed PV- Loop 9082', # VPSA p-001 col DC
    'RNGDGTS-VPSA2 Vac Pump P002 Speed PV- Loop 9102', # VPSA p-002 col DD

]]


roflo_speed_perc = np.where(combined_df['RNGDGTS-Plant compressor Speed- Loop 8964.2'] > 250, ( combined_df['RNGDGTS-Plant compressor Speed- Loop 8964.2']/760) * 100, 0)

vpsa_graph_df = pd.DataFrame(vpsa_graph)

vpsa_graph_df['roflo_speed_perc'] = roflo_speed_perc

vpsa_graph_df.columns = [
    'Start Time', 
    'VPSA status',
    'VPSA stage 1 off-gas flow',
    'VPSA stage 2 off-gas flow',
    'VPSA P-001',
    'VPSA P-002',
    'Totara Speed (%)'
]

vpsa_graph_df.to_excel('vpsa_graph_df.xlsx')

### RNGF AIT 500


In [ ]:
# add this
roflo_speed_perc # Totara Speed (%) col AT

rngf_graph = combined_df[[
    'Start Time',
    'RNGDGTS-Biogas Stream CH4- Loop 8942', # Biogas CH4 col Y,
    'RNGDGTS-Biomethane Stream CH4- Loop 9072', # Product CH4 col AD
    'RNGDGTS-NOVA Stage 1 CH4- Loop 9056', # NOVA CH4 col AJ
    'RNGDGTS-Biogas Stream CO2- Loop 8943', # Biogas CO2 col Z
    'RNGDGTS-Biomethane Stream CO2- Loop 9075', # Product CO2 col AE
    'RNGDGTS-VPSA Stage 1 Running- Loop 9035', # VPSA status col CW
    'RNGDGTS-VPSA1 Product Gas to Flare- Loop 9078' # Product to Flare (CV -120) col DB
]]

rngf_graph_df = pd.DataFrame(rngf_graph)
rngf_graph_df.columns = [
    'date/time',
    'Biogas CH4',
    'Product CH4',
    'NOVA CH4',
    'Biogas CO2',
    'Product CO2',
    'VPSA Status',
    'Product to Flare (CV-120)'
    'Totara speed (%)'
]


rngf_graph_df['Totara speed (%)'] = roflo_speed_perc

rngf_graph_df.to_excel('rngf_graph_df.xlsx')



### RNGF Unison Blower Overview

In [ ]:
unison_blower = combined_df[[
    'Start Time',
    'RNGBLWR-Skid Disch FLOW PV- Loop 9159', # Blower discharge flow (scfm) col F
    'RNGBLWR-System Gas Disch PRESS PV- Loop 9158', # blower discharge press (psig) col G
    'RNGBLWR-Bstr Blwr 1 Speed FDBK PV- Loop 9145', # unison blower 1 speed (%) col C
    'RNGBLWR-Bstr Blwr 2 Speed FDBK PV- Loop 9149', # unison blower 2 speed (%)
    'RNGDGTS-Plant compressor Speed- Loop 8964', # Totara speed (rpm) col N
]]

unison_blower_df = pd.DataFrame(unison_blower)
unison_blower_df.columns = [
    'date/time',
    'Blower discharge flow (scfm)',
    'blower discharge press (psig)',
    'unison blower 1 speed (%)',
    'unison blower 2 speed (%)',
    'Totara speed (rpm)'
]

unison_blower_df.to_excel('unison_blower_df.xlsx')


### RTO Overview


In [ ]:


rto_overview = combined_df[[
    'Start Time',
    'RNGDGTS-Plant compressor Speed- Loop 8964', # Totara speed (rpm) col N
    'RNGRTO-Iso Valve Opened- Loop 9182', # Isolation Valve col V
    'RNGRTO-Gas Trtmnt Tail Gas Flow PV- Loop 9105', # RTO inlet flow (scfm) col T
    'RNGRTO-Inlet PRESSURE PV- Loop 9181', # RTO inlet pressure (WC) col U
    'RNGRTO-Process Air LEL PV- Loop 9180', # RTO LEL (%) col CC
]]

rto_overview_df = pd.DataFrame(rto_overview)
rto_overview_df.columns = [
    'date/time',
    'Totara speed (rpm)',
    'Isolation valve',
    'RTO inlet flow (scfm)',
    'RTO inlet pressure (WC)',
    'RTO LEL (%)'
]

rto_overview_df.to_excel('rto_overview_df.xlsx')

### O2 N2 and H2S

In [ ]:
AIT_500_gases = combined_df[[
    'Start Time',
    'RNGDGTS-VPSA Stage 1 Running- Loop 9035', # VPSA Status col CW
    'RNGDGTS-VPSA1 Product Gas to Flare- Loop 9078', # VPSA to Flare (CV-120) col DB
    'RNGDGTS-Biogas Stream O2- Loop 8944', # Biogas O2 col AC
    'RNGDGTS-Biomethane Stream O2- Loop 9073', # Product O2 col AH
    'RNGDGTS-Biogas Stream N2- Loop 8945', # Biogas N2 col AB
    'RNGDGTS-Biomethane Stream N2- Loop 9074', # Product N2
    'RNGDGTS-Biogas Stream H2S- Loop 8946', # Biogas H2S col AA
    'RNGDGTS-Biomethane Stream H2S- Loop 9076', # Product H2S col AF
]]


AIT_500_gases_df = pd.DataFrame(AIT_500_gases)
AIT_500_gases_df['Totara speed (%)'] = roflo_speed_perc

AIT_500_gases_df.columns = [
    'date/time',
    'VPSA Status',
    'VPSA to Flare (CV-120)',
    'Biogas O2',
    'Product O2',
    'Biogas N2',
    'Product N2',
    'Biogas H2S',
    'Product H2S',
    'Totara speed (%)',
    # 
]

AIT_500_gases_df.to_excel('AIT_500_gases_df.xlsx')

### ANGI Compressor

In [ ]:
angi_compressor = combined_df[[
    'Start Time',
    'RNGCOMP-Stage 1 Inlet Press PV- Loop 9234', # ANGI S1 inlet (psig) col I
    # ANGI speed (%) ANGI_speed
    # Totara speed (%) roflo_speed_perc
    'RNGDGTS-VPSA Stage 1 Running- Loop 9035', # VPSA Status col DE
    'RNGDGTS-VPSA1 Product Gas to Flare- Loop 9078', # DGTS to Flare col DF
    'RNGCOMP-Stage 2 Final Press PV- Loop 9241', # ANGI S2 discharge (psig) col J
    
]]

roflo_speed_perc
ANGI_speed = np.where(combined_df['RNGCOMP-Comp Speed PV- Loop 9250'] > 800, (combined_df['RNGCOMP-Comp Speed PV- Loop 9250'] / 1800) * 100, 0)  

angi_compressor_df = pd.DataFrame(angi_compressor)

angi_compressor_df['roflo_speed_perc'] = roflo_speed_perc
angi_compressor_df['ANGI_speed'] = ANGI_speed

angi_compressor_df.columns = [
    'date/time',
    'ANGI S1 inlet (psig)',
    'VPSA status',
    'DGTS to flare',
    'ANGI S2 discharge',
    'roflo speed (%)',
    'ANGI speed'
]


angi_compressor.to_excel('ANGI_compressor_df.xlsx')

### Flows

In [ ]:
flows = combined_df[[
    'Start Time',
    'RNGBLWR-Skid Disch FLOW PV- Loop 9159', # Unison discharge flow (scfm) col F
    'RNGDGTS-Boigas Stream Flow- Loop 8940', # Totara inlet (scfm) col K
    'RNGDGTS-Gas flow- Loop 9069.1', # DGTS Product gas flow (scfm) col M
    'RNGFLARE-Reject Gas to Flare Flow PV- Loop 9316', # Flare flow (scfm)
    # Totara speed (%) roflo_speed_perc
]]

flows_df = pd.DataFrame(flows)
flows_df['totara speed (%)'] = roflo_speed_perc

flows_df.columns = [
    'date/time',
    'Unison discharge flow (scfm)',
    'Totara inlet (scfm)',
    'DGTS product gas flow',
    'Flare flow',
    'totara speed (%)'
]

flows_df.to_excel('flows_df.xlsx')